In [1]:
import numpy as np
import itertools
import math
import time

# Task 1: Unique non-flat vector search

In [2]:
task1 = "Unique non-flat vector search"

In [3]:
def is_flat(x):
    if len(x) == 0:
        return True
    else:
        return (x == x[0]).all()
    
assert is_flat(np.array([1,1,1]))
assert not is_flat(np.array([1.1,1,1]))
assert is_flat(np.array([]))

In [4]:
def get_unique_non_flat_vectors(matrix):
    unique = set()
    result = []
    for i, column in enumerate(matrix.T):
        tuple_column = tuple(column)
        if not is_flat(column) and tuple_column not in unique:
            unique.add(tuple_column)
            result.append(i + 1)
    
    return result

In [5]:
print(f'{task1} - testing case from assignment')
assert (get_unique_non_flat_vectors(np.array(
    [
        [1.0, 2.0, 1.0, 1.2, 1.9],
        [1.0, 2.0, 1.0, 1.2, -.9],
        [3.0, 2.0, 3.0, 3.2, -.6],
        [1.0, 2.0, 1.0, 1.2, -0.9],
        [1.0, 2.0, 1.0, 2.2, -0.9],
        [1.0, 2.0, 1.0, 1.2, 0.1],
    ]
)) == [1,4,5])

print(f'{task1} - testing with all flat vectors')
assert (get_unique_non_flat_vectors(np.array(
    [
        [1.0, 1.1, 1.0],
        [1.0, 1.1, 1.0],
    ]
)) == [])

print(f'{task1} - testing 0x0 matrix input')
assert (get_unique_non_flat_vectors(np.array(
    [[]]
)) == [])


N = 5000 # rows
M = 10000 # columns

large_matrix_with_each_5th_column_unique_nonflat = np.array([
    np.append(np.array(i), np.random.randn(N-1)) if i % 5 == 0 else np.ones(N) * i
    for i in range(M)
]).T

large_matrix_with_each_5th_column_unique_nonflat_indeces = [i + 1 for i in range(M) if i % 5 == 0]

t0 = time.time()
assert get_unique_non_flat_vectors(large_matrix_with_each_5th_column_unique_nonflat) == large_matrix_with_each_5th_column_unique_nonflat_indeces
t1 = time.time()

print(f'{task1} - testing {N}x{M} matrix input, took {t1-t0:.4f} seconds')


Unique non-flat vector search - testing case from assignment
Unique non-flat vector search - testing with all flat vectors
Unique non-flat vector search - testing 0x0 matrix input
Unique non-flat vector search - testing 5000x10000 matrix input, took 3.6884 seconds


# Task 2: Modified Gram–Schmidt subroutine

In [6]:
task2 = "Modified Gram–Schmidt subroutine"

In [7]:
def proj(u, v):
    if np.linalg.norm(u) == 0:
        return u
    else:
        return u.dot(v) / u.dot(u) * u

assert (proj(np.array([1,0]), np.array([3,3])) == np.array([3,0])).all()
assert (proj(np.array([0,0]), np.array([3,3])) == np.array([0,0])).all()

In [8]:
def is_orthonormal(matrix):
    columns = (column for column in matrix.T)

    for c in columns:
        norm = np.linalg.norm(c)
        if not math.isclose(norm, 1) and not math.isclose(norm, 0):
            return False
    
    for c1, c2 in itertools.combinations(columns, 2):
        if c1.dot(c2) != 0:
            return False
        
    return True
    
matrix_orthonormal_a = np.array([
    [1],
    [1],
])

matrix_orthonormal_b = np.array([
    [1, 0],
    [0, 1],
])

matrix_orthonormal_c = np.array([
    [2, 0],
    [0, 1],
])

assert is_orthonormal(matrix_orthonormal_a) == False
assert is_orthonormal(matrix_orthonormal_b) == True
assert is_orthonormal(matrix_orthonormal_c) == False

In [9]:
def mod_graham_schmidt(matrix):
    columns = [column for column in matrix.T]
    
    results = []
    for i in range(len(columns)):
        #print(i)
        v = columns[i]
        columns_rest = columns[i+1:]
        
        for u in columns_rest:
            v = v - proj(u, v)
        
        v = v / np.linalg.norm(v) if np.linalg.norm(v) else v
        
        results.append(v)
        #print(results)
        
    return np.array(results).T
    
        
A = np.array([
    [2,0],
    [0,2]
]).T

A_res = np.array([
    [1,0],
    [0,1],
])
    
assert (A_res == mod_graham_schmidt(A)).all()
assert is_orthonormal(mod_graham_schmidt(A))
print(f'{task2} - testing simple case')

Modified Gram–Schmidt subroutine - testing simple case


In [10]:
B_1 = np.array([1,2,3])
B_2 = np.array([-1,0,1])
B = np.array([
    B_1, B_2, B_1 + B_2 + np.array([0,1,0])
]
).T

mgsB = mod_graham_schmidt(B)
assert is_orthonormal(mgsB)
print(f'{task2} - testing non independent vectors')

Modified Gram–Schmidt subroutine - testing non independent vectors


In [11]:
C = np.array([
    [1,0,1],
    [0,1,1],
    [0,0,0],
])


C_orthogonalized = mod_graham_schmidt(C)

print(f'{task2} - Test that if a span is overdefined, that at least one of the columns has to be eliminted')

assert is_orthonormal(C_orthogonalized)

Modified Gram–Schmidt subroutine - Test that if a span is overdefined, that at least one of the columns has to be eliminted


In [12]:
N1 = 5000
M1 = 1000

large1 = np.array([np.random.randn(N1) for i in range(M1)]).T

N2 = 5000
M2 = 2000

large2 = np.array([np.random.randn(N2) for i in range(M2)]).T

t0 = time.time()
large1_orthonormal = mod_graham_schmidt(large1)
t1 = time.time()
assert is_orthonormal(large1_orthonormal)
print(f'{task2} - testing {N1}x{M1}, took {t1-t0:.2f} second')

t0 = time.time()
large2_orthonormal = mod_graham_schmidt(large2)
t1 = time.time()
assert is_orthonormal(large2_orthonormal)

print(f'{task2} - testing {N2}x{M2}, took {t1-t0:.2f} second')

Modified Gram–Schmidt subroutine - testing 5000x1000, took 10.96 second
Modified Gram–Schmidt subroutine - testing 5000x2000, took 32.61 second


# T statistic calculation

In [13]:
task3 = 'T statistic calculation'

In [14]:
def calc_T(arr):
    N = len(arr)
    arr_avg = arr.mean()
    denominator_array = arr[1:] - arr[:-1]
    numerator_array = arr - arr_avg
    result = (1/(N-1))/(1/N) * sum(denominator_array ** 2) / sum(numerator_array ** 2)
    
    return result

In [15]:
assert calc_T(np.array([1,2,6])) == (1/(3-1)) / (1/3) *  (1**2 + 4**2) / (2**2 + 1**2 + 3**2)
print(f'{task3} - testing simple case')

N = 10**7
t0 = time.time()
value =  calc_T(np.random.randn(N))
t1 = time.time()

print(f'{task3} - series of length {N:,} took {t1-t0:.2f} seconds')

T statistic calculation - testing simple case
T statistic calculation - series of length 10,000,000 took 3.08 seconds
